In [ ]:
import os
import numpy as np
import nibabel as nib
import SimpleITK as sitk
from scipy.ndimage import label
import subprocess
import matplotlib.pyplot as plt


root_directory = "./derivatives/"


def load_nifti(file_path):
    """Carica un volume MRI in formato NIfTI e lo converte in un array NumPy."""
    nifti_img = nib.load(file_path)
    volume = nifti_img.get_fdata()
    return volume, nifti_img


def get_components(image, min_size, max_size, full_connectivity=True):
    """Extract connected components and compute lesion volumes."""
    ccifilter = sitk.ConnectedComponentImageFilter()
    ccifilter.SetFullyConnected(full_connectivity)  # 26-connectivity if True, else 6-connectivity
    labeled = ccifilter.Execute(image)
    rcif = sitk.RelabelComponentImageFilter()
    rcif.SetMinimumObjectSize(min_size) 
    #rcif.SetMaximumObjectSize(max_size)
    labeled = rcif.Execute(labeled)
    #print(labeled)
    labeled_data = sitk.GetArrayFromImage(labeled)
    
    # Get unique labels and their counts
    labels, counts = np.unique(labeled_data, return_counts=True)

    # Find labels that occur > 30 times
    rare_labels = labels[counts > 30]

    for lbl in rare_labels:
        labeled_data[labeled_data == lbl] = 0

    ncomponents = rcif.GetNumberOfObjects()

    """if ncomponents != ccifilter.GetObjectCount():
        print('Number of components mismatch!')
        print(ccifilter.GetObjectCount())"""

    # Compute lesion volumes
    lsif = sitk.LabelStatisticsImageFilter()
    lsif.Execute(labeled, labeled)

    spacing = image.GetSpacing()  # Get voxel spacing (dx, dy, dz)
    voxel_volume = spacing[0] * spacing[1] * spacing[2]  # Volume of a single voxel in mm³

    lesion_volumes = []
    for label in range(1, ncomponents + 1):  # Labels start from 1
        voxel_count = lsif.GetCount(label)  # Number of voxels in this lesion
        volume_mm3 = voxel_count * voxel_volume  # Convert to mm³
        lesion_volumes.append((label, voxel_count, volume_mm3))

    return labeled_data, ncomponents, lesion_volumes



def preprocess_gt(gt_file, output_file, threshold=0.5, min_size_mm3=10, max_size_mm3=30):
    """Pipeline per preprocessare il GT."""
    #volume, nifti_img = load_nifti(gt_file)
    #voxel_spacing = nifti_img.header.get_zooms()
    
    #num_lesions, labeled_volume, lesion_volumes = count_26_connectivity_components(volume, voxel_spacing, 0)
    #print(f"Processed GT saved at: {output_file}")
    #print(f"Numero di lesioni finali: {num_lesions}")
    #print("Volumi delle lesioni in mm³:")
    #for lesion, vol in lesion_volumes.items():
    #    print(f" - Lesione {lesion}: {vol:.2f} mm³")
    image_T1 = sitk.ReadImage(gt_file, sitk.sitkFloat64)

# Convert to binary
    image_T1 = sitk.BinaryThreshold(image_T1,
                                lowerThreshold=0.5, 
                                upperThreshold=1e6, 
                                insideValue=1, 
                                outsideValue=0)

    # Process the image
    min_size = min_size_mm3
    max_size = max_size_mm3
    labeled_data, len_lesion_volumes, lesion_volumes = get_components(image_T1, min_size=10, max_size=30, full_connectivity=True)

    # Convert back to a SimpleITK image
    filtered_image = sitk.GetImageFromArray(labeled_data)
    filtered_image.CopyInformation(image_T1)  # Preserve original image metadata

    # Save the new image
    #output_path = gt_file.replace('.nii.gz', '_prep.nii.gz')
    sitk.WriteImage(filtered_image, output_file)

    print(f"Filtered image saved at: {output_file}")
    return labeled_data



In [8]:
subjects = ['sub-r001s006', 'sub-r001s015', 'sub-r001s016', 'sub-r001s023',
 'sub-r002s003', 'sub-r002s006', 'sub-r003s010', 'sub-r004s002',
 'sub-r004s005', 'sub-r004s006', 'sub-r004s007', 'sub-r004s010',
 'sub-r004s016', 'sub-r004s019', 'sub-r004s025', 'sub-r005s045',
 'sub-r005s074', 'sub-r009s003', 'sub-r009s004', 'sub-r009s008',
 'sub-r009s013', 'sub-r009s017', 'sub-r009s027', 'sub-r009s034',
 'sub-r009s039', 'sub-r009s044', 'sub-r009s051', 'sub-r009s052',
 'sub-r009s060', 'sub-r009s077', 'sub-r009s092', 'sub-r010s014',
 'sub-r010s027', 'sub-r011s011', 'sub-r011s014', 'sub-r015s011',
 'sub-r017s104', 'sub-r017s110', 'sub-r017s119', 'sub-r023s003',
 'sub-r023s009', 'sub-r024s002', 'sub-r024s018', 'sub-r024s019',
 'sub-r029s004', 'sub-r031s001', 'sub-r031s003', 'sub-r031s004',
 'sub-r031s007', 'sub-r031s010', 'sub-r031s011', 'sub-r031s017',
 'sub-r031s019', 'sub-r031s025', 'sub-r034s006', 'sub-r034s015',
 'sub-r034s022', 'sub-r038s020', 'sub-r040s043', 'sub-r040s078',
 'sub-r042s003', 'sub-r042s011', 'sub-r042s023', 'sub-r042s025',
 'sub-r048s002', 'sub-r048s015', 'sub-r048s022', 'sub-r048s035', 
 'sub-r052s001', 'sub-r052s021']



for subject in subjects:
    subject_path = os.path.join(root_directory, subject)
    seg_folder = os.path.join(subject_path, "seg")
    print(seg_folder)
    labeled_data = None
    for file in os.listdir(seg_folder):
        if file.endswith("seg.nii.gz"):
            file_path = os.path.join(seg_folder, file)
            output_gt = file_path.replace('.nii.gz', '_prep10_30.nii.gz')
            print(f"Processing {file_path}")
            labeled_data = preprocess_gt(file_path, output_gt)


./derivatives/sub-r001s006/seg
Processing ./derivatives/sub-r001s006/seg/sub-r001s006_seg.nii.gz
Filtered image saved at: ./derivatives/sub-r001s006/seg/sub-r001s006_seg_prep10_30.nii.gz
./derivatives/sub-r001s015/seg
Processing ./derivatives/sub-r001s015/seg/sub-r001s015_seg.nii.gz
Filtered image saved at: ./derivatives/sub-r001s015/seg/sub-r001s015_seg_prep10_30.nii.gz
./derivatives/sub-r001s016/seg
Processing ./derivatives/sub-r001s016/seg/sub-r001s016_seg.nii.gz
Filtered image saved at: ./derivatives/sub-r001s016/seg/sub-r001s016_seg_prep10_30.nii.gz
./derivatives/sub-r001s023/seg
Processing ./derivatives/sub-r001s023/seg/sub-r001s023_seg.nii.gz
Filtered image saved at: ./derivatives/sub-r001s023/seg/sub-r001s023_seg_prep10_30.nii.gz
./derivatives/sub-r002s003/seg
Processing ./derivatives/sub-r002s003/seg/sub-r002s003_seg.nii.gz
Filtered image saved at: ./derivatives/sub-r002s003/seg/sub-r002s003_seg_prep10_30.nii.gz
./derivatives/sub-r002s006/seg
Processing ./derivatives/sub-r002s

In [9]:
import os
import shutil


# Folder where the source files are stored
source_folder = './derivatives/'

# Root folder where subject folders are located
destination_root = 'derivatives_10_30_mm'
os.makedirs(destination_root, exist_ok=True)
# Loop and copy each subject's corresponding file
for subject in subjects:
    filename = f"{subject}/seg/{subject}_seg_prep10_30.nii.gz"
    src = os.path.join(source_folder, filename)
    dst_folder = destination_root
    dst = destination_root

    if not os.path.exists(src):
        print(f"❌ File not found: {src}")
    elif not os.path.exists(dst_folder):
        print(f"⚠️  Destination folder does not exist: {dst_folder}")
    else:
        shutil.copy(src, dst)
        print(f"✅ Copied {filename} to {dst_folder}")


✅ Copied sub-r001s006/seg/sub-r001s006_seg_prep10_30.nii.gz to derivatives_10_30_mm
✅ Copied sub-r001s015/seg/sub-r001s015_seg_prep10_30.nii.gz to derivatives_10_30_mm
✅ Copied sub-r001s016/seg/sub-r001s016_seg_prep10_30.nii.gz to derivatives_10_30_mm
✅ Copied sub-r001s023/seg/sub-r001s023_seg_prep10_30.nii.gz to derivatives_10_30_mm
✅ Copied sub-r002s003/seg/sub-r002s003_seg_prep10_30.nii.gz to derivatives_10_30_mm
✅ Copied sub-r002s006/seg/sub-r002s006_seg_prep10_30.nii.gz to derivatives_10_30_mm
✅ Copied sub-r003s010/seg/sub-r003s010_seg_prep10_30.nii.gz to derivatives_10_30_mm
✅ Copied sub-r004s002/seg/sub-r004s002_seg_prep10_30.nii.gz to derivatives_10_30_mm
✅ Copied sub-r004s005/seg/sub-r004s005_seg_prep10_30.nii.gz to derivatives_10_30_mm
✅ Copied sub-r004s006/seg/sub-r004s006_seg_prep10_30.nii.gz to derivatives_10_30_mm
✅ Copied sub-r004s007/seg/sub-r004s007_seg_prep10_30.nii.gz to derivatives_10_30_mm
✅ Copied sub-r004s010/seg/sub-r004s010_seg_prep10_30.nii.gz to derivatives_1

In [20]:
files_10_30 = '/home/francesca/Desktop/post_stroke_project/ATLAS_Dataset/derivatives_10_30_mm'

In [21]:
import nibabel as nib
import os
for file in os.listdir(files_10_30):
    #print(file)
    print(file, np.unique(nib.load(f'derivatives_10_30_mm/{file}').get_fdata(), return_counts=True))
    


sub-r001s015_seg_prep10_30.nii.gz (array([0., 2.]), array([8675261,      28]))
sub-r017s110_seg_prep10_30.nii.gz (array([0., 2.]), array([8675269,      20]))
sub-r023s003_seg_prep10_30.nii.gz (array([0., 6., 7.]), array([8675258,      20,      11]))
sub-r031s011_seg_prep10_30.nii.gz (array([ 0., 10.]), array([8675263,      26]))
sub-r042s003_seg_prep10_30.nii.gz (array([0., 2.]), array([8675276,      13]))
sub-r017s104_seg_prep10_30.nii.gz (array([0., 6.]), array([8675273,      16]))
sub-r003s010_seg_prep10_30.nii.gz (array([0., 3.]), array([8675270,      19]))
sub-r029s004_seg_prep10_30.nii.gz (array([0., 7.]), array([8675269,      20]))
sub-r040s078_seg_prep10_30.nii.gz (array([0., 2.]), array([8675271,      18]))
sub-r001s023_seg_prep10_30.nii.gz (array([0., 1.]), array([8675274,      15]))
sub-r031s003_seg_prep10_30.nii.gz (array([0., 4.]), array([8675275,      14]))
sub-r009s051_seg_prep10_30.nii.gz (array([0., 4.]), array([8675266,      23]))
sub-r031s007_seg_prep10_30.nii.gz (ar

In [22]:
import nibabel as nib

for file in os.listdir(files_10_30):
    print(np.unique(nib.load(f'derivatives_10_30_mm/{file}').get_fdata(), return_counts=True))
    


(array([0., 2.]), array([8675261,      28]))
(array([0., 2.]), array([8675269,      20]))
(array([0., 6., 7.]), array([8675258,      20,      11]))
(array([ 0., 10.]), array([8675263,      26]))
(array([0., 2.]), array([8675276,      13]))
(array([0., 6.]), array([8675273,      16]))
(array([0., 3.]), array([8675270,      19]))
(array([0., 7.]), array([8675269,      20]))
(array([0., 2.]), array([8675271,      18]))
(array([0., 1.]), array([8675274,      15]))
(array([0., 4.]), array([8675275,      14]))
(array([0., 4.]), array([8675266,      23]))
(array([0., 3.]), array([8675268,      21]))
(array([0., 6., 7.]), array([8675244,      30,      15]))
(array([0., 2.]), array([8675270,      19]))
(array([ 0., 15., 16., 17., 18., 19., 20., 21., 22., 23.]), array([8675097,      29,      25,      24,      23,      20,      20,
            19,      19,      13]))
(array([0., 2.]), array([8675274,      15]))
(array([ 0.,  9., 10., 11., 12.]), array([8675206,      28,      20,      18,      17]